In [65]:
import pandas, numpy, os, re

In [66]:
article_directory = './hw01-data-raw/News Articles'
article_list = os.listdir(article_directory) # create list of files
article_list[:5]

['1101243072871.txt',
 '1101163341619.txt',
 '1101163941020.txt',
 '1101163268260.txt',
 '1101163374885.txt']

In [67]:
keywords = { 
    'mad cow': ['mad cow', 'prion', 'creutzfeldt-jakob', 'spongiform', 'encephalopath'], 
    'laboratory': ['lab'],
    'whistleblower': ['whistle'],
    'key names': ['Ryker', 'Sanderson', 'Boynton', 'Dortmund', 'Sulfate'],
    'Laurie/Laurel Sulfate': ['Sulfate', 'Laurie', 'Laurel'],
    'animal feed': ['feed'],
    'cash flow': ['$', 'dollar', 'money', 'cash', 'expense'],
    'mayor': ['locke']
}
keywords.keys()

dict_keys(['mad cow', 'laboratory', 'whistleblower', 'key names', 'Laurie/Laurel Sulfate', 'animal feed', 'cash flow', 'mayor'])

regular expression from https://stackoverflow.com/questions/17268958/finding-occurrences-of-a-word-in-a-string-in-python-3

In [68]:
article_data = []
# create dataframe from list of dictionaries
# each dictionary: {filename: filename.txt, title: "", author: {author name}, date: {date posted}, ...}

for file_name in article_list:
    # open the file
    article = open(f'{article_directory}/{file_name}', encoding='ISO 8859-1')
    article_text = article.read()
    # replace weird characters and newlines with single space, convert to lower case
    # article_text = article_text.replace('\x97', ' ').replace('\xa0', ' ').replace('\n', ' ')
    # article_text = re.sub(' +', ' ', article_text).lower()

    # empty list of article keywords to populate
    article_keywords = []
    article_dict = { 'filename': file_name}
    # get other metadata for article_dict from Emi's code (article title, author, date published)
    article_text = article_text.replace('\x97', ' ').replace('\xa0', ' ').replace('\n', '#')
    article_text = re.sub(' +', ' ', article_text).lower()
    article_text = re.sub('(#)+', '#', article_text)
    article_text = article_text[1:]
    article_pieces =  article_text.split('#')
    # print(article_pieces)
    title = article_pieces[0]
    author = False
    date = False
    if "story by:" in article_pieces[1]:
        author = article_pieces[1][10:]
        date = article_pieces[2][23:]
    elif article_pieces[0] == " ":
        date = article_pieces[1].strip()[23:]
    else:
        date = article_pieces[2].strip()[23:]
    article_dict['title'] = title
    article_dict['date'] = date
    article_dict['author'] = author
    # tally up occurrences of each keyword category
    tally = 0
    for keyword in keywords.keys():
        article_dict[keyword] = 0
        contains_keyword = False
        for synonym in keywords[keyword]:
            num_found = sum(1 for _ in re.finditer(r'%s' % re.escape(synonym.lower()), article_text))
            article_dict[keyword] += num_found
            tally+=num_found
    if tally>0:
        article_data.append(article_dict)

In [69]:
article_df = pandas.DataFrame(article_data)
article_df['date'] = pandas.to_datetime(article_df['date'], format='mixed')
# article_df.sort_values(by='key names', ascending=False)
param = 'mayor'
article_df[article_df[param] > 0].sort_values(by='date')

,filename,title,date,author,mad cow,laboratory,whistleblower,key names,Laurie/Laurel Sulfate,animal feed,cash flow,mayor
93,1101243489577.txt,seeger continues pressuring council to open co...,2002-01-08,john panni,0,0,0,0,0,0,1,2
54,1101243423983.txt,firefighter takes exception to councilman's co...,2002-01-20,False,0,0,0,0,0,0,2,1
533,1101243338500.txt,uthor announces biotech lab to locate in alder...,2002-02-02,mel sacher,4,8,0,6,2,0,4,2
547,1101243338264.txt,alderwood city council briefs,2002-02-03,False,0,0,0,0,0,0,1,2
314,1101243358045.txt,graffiti crackdown on feb. 9 agenda,2002-02-03,False,0,0,0,0,0,0,1,2
303,1101243337060.txt,court ruling reverses annexation law in state,2002-02-05,melissa browning,0,0,0,0,0,0,0,4
85,1101243227372.txt,council touches up graffiti ordinance,2002-03-02,john panni,0,0,0,0,0,0,2,8
515,1101163409838.txt,city leaders conservatively ete new housing re...,2002-04-12,john panni,0,0,0,0,0,0,4,1
395,1101163870045.txt,city council withdraws support of outreach pro...,2003-01-13,john panni,0,0,0,0,0,0,8,2
40,1101163836903.txt,city council opts to make collective bargainin...,2003-01-20,john panni,0,1,0,0,0,0,2,1
